In [1]:
import copy, sys
import random, numpy as np

sys.path.append("../../../")
from importlib import reload

import inferring.InferringDFA as InferringDFA
import utils.automata.DFA
import utils.advice_systems.SRS as SRS
import utils.runLearnLib.RunLearnLib as RunLearnLib
import utils.display.tables as Tables

reload(InferringDFA)
reload(RunLearnLib)
reload(utils.automata.DFA)
reload(SRS)
reload(Tables)

from inferring.InferringDFA import InferringDFA
from utils.runLearnLib.RunLearnLib import RunLearnLib
from utils.automata.DFA.DFA import DFA
from utils.advice_systems.SRS import SRS
from utils.display.tables import Tables

In [2]:
class Record:
    def __init__(self,
                 dfa_sz,
                 lstar,
                 lstar_as, 
                 target=None):
        self.dfa_sz = dfa_sz
        self.lstar = lstar
        self.lstar_as = lstar_as
        self.target = target

    def print_record(self):
        print(
            f"|Q| = {self.dfa_sz} eq = {self.lstar[1]}, eq_with_advice = {self.lstar_as[1]}"
        )

## Idempotent letter

In this experiment, we learn languages in which one letter is idempotent, using the LearnLib implementation of L*.

In [3]:
# Fixed random seeds for reproducibility
number_of_iteration = 100
seeds = [i for i in range(number_of_iteration)]

# The bound on the number of states
max_number_of_states = 1000

# The alphabet of the DFA
input_signs = ['a', 'b', 'c', 'd']

# Structure to aggregate the results.
results = []

# Build the necessary part of the learnLib project.
helper = RunLearnLib()
helper.compileLearnLib()

for i, seed in enumerate(seeds):
    random.seed(seed)
    # Create a random DFA with an idempotent letter and run the learning process to minimize it.
    # Exclude the degenerate cases (DFA of size smaller than 2).
    dfa = DFA()
    while dfa.Q < 2:
        d = DFA()
        d.create_random_indempotent_automaton(Q=random.randint(
            max_number_of_states // 2, max_number_of_states),
                                              input_signs=input_signs,
                                              letter=input_signs[0])
        learn_dfa = InferringDFA(target=copy.deepcopy(d))
        dfa, learning_info = learn_dfa.run()

    d.save_complete_description()

    # Learn language of d automaton without advice, using LearnLib implementation of L*.
    lstar_output = helper.runLstar()
    # Learn language of d automaton with advice, using LearnLib implementation of L*.
    lstar_as_output = helper.runLstarIdemWithAdvice()

    results.append(
        Record(dfa_sz=d.Q,
               lstar=lstar_output,
               lstar_as=lstar_as_output,
               target=d))

    print(f"iter {i} done")

iter 0 done
iter 1 done
iter 2 done
iter 3 done
iter 4 done
iter 5 done
iter 6 done
iter 7 done
iter 8 done
iter 9 done
iter 10 done
iter 11 done
iter 12 done
iter 13 done
iter 14 done
iter 15 done
iter 16 done
iter 17 done
iter 18 done
iter 19 done
iter 20 done
iter 21 done
iter 22 done
iter 23 done
iter 24 done
iter 25 done
iter 26 done
iter 27 done
iter 28 done
iter 29 done
iter 30 done
iter 31 done
iter 32 done
iter 33 done
iter 34 done
iter 35 done
iter 36 done
iter 37 done
iter 38 done
iter 39 done
iter 40 done
iter 41 done
iter 42 done
iter 43 done
iter 44 done
iter 45 done
iter 46 done
iter 47 done
iter 48 done
iter 49 done
iter 50 done
iter 51 done
iter 52 done
iter 53 done
iter 54 done
iter 55 done
iter 56 done
iter 57 done
iter 58 done
iter 59 done
iter 60 done
iter 61 done
iter 62 done
iter 63 done
iter 64 done
iter 65 done
iter 66 done
iter 67 done
iter 68 done
iter 69 done
iter 70 done
iter 71 done
iter 72 done
iter 73 done
iter 74 done
iter 75 done
iter 76 done
iter 77 d

In [4]:
results.sort(key=lambda x: (x.dfa_sz))

In [5]:
lstar_eqs = [r.lstar[1] for r in results]
lstar_as_eqs = [r.lstar_as[1] for r in results]

red = (
    (np.array(lstar_eqs) - np.array(lstar_as_eqs)) / np.array(lstar_eqs)) * 100
columns = [('Target language', 'DFA'), ('L*', 'EQ'), ('L* with advice', 'EQ'), ('Reduction', 'EQ')]
data = {
    columns[0]: [r.dfa_sz for r in results],
    columns[1]: lstar_eqs,
    columns[2]: lstar_as_eqs,
    columns[3]: [int(r) for r in red]
}

table_creator = Tables()
df = table_creator.create_basic_table(columns=columns, data=data)
display(df)

,Target language,L*,L* with advice,Reduction
,DFA,EQ,EQ,EQ
0,506,48,41,14%
1,519,53,32,39%
2,521,41,35,14%
3,534,45,30,33%
4,536,41,31,24%
...,...,...,...,...
99,1000,49,38,22%
100,,,,
101,Max reduction,,,55%


In [9]:
import pandas as pd 
with pd.option_context('display.max_colwidth', None, 'display.max_rows', None): 
    display(df)

,Target language,L*,L* with advice,Reduction
,DFA,EQ,EQ,EQ
0,506,48,41,14%
1,519,53,32,39%
2,521,41,35,14%
3,534,45,30,33%
4,536,41,31,24%
5,537,56,34,39%
6,538,51,40,21%
7,539,42,33,21%
8,541,51,32,37%
